In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import operator
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()

generatorModel = ChatOpenAI(model='gpt-4o-mini')
evaluatorModel = ChatOpenAI(model='gpt-4o')
optimizerModel = ChatOpenAI(model='gpt-4o-mini')

In [15]:
class tweetState(TypedDict):
  topic: str
  tweet: str
  evaluation: Literal["approved", "needsImprovement"]
  feedback: str
  iteration: int
  maxIteration: int

In [6]:
class tweetEvaluation(BaseModel):
  evaluation: Literal['approved', 'needsImprovement'] = Field(..., description="Final evaluation result.")
  feedback: str = Field(..., description="Constructive feedback for the tweet")

structuredEvaluationModel = evaluatorModel.with_structured_output(tweetEvaluation)

In [7]:
def generateTweet(state: tweetState):
  messages = [
  SystemMessage(content="You are a funny and clever Twitter/X influencer."),
  HumanMessage(content=f""" Write a short, original, and hilarious tweet on the topic: "{state['topic']}". 
               Rules - Do NOT use question-answer format. 
               - Max 280 characters. 
               - Use observational humor, irony, sarcasm, or cultural references. 
               - Think in meme logic, punchlines, or relatable takes. 
               - Use simple, day to day english 
               - This is version {state['iteration'] + 1}.
  """)
  ]

  tweet = generatorModel.invoke(messages).content

  return {'tweet':tweet}


def evaluateTweet(state: tweetState):
  messages = [
  SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
  HumanMessage(content=f"""
    Evaluate the following tweet:

    Tweet: "{state['tweet']}"

    Use the criteria below to evaluate the tweet:
      1. Originality - Is this fresh, or have you seen it a hundred times before?
      2. Humor - Did it genuinely make you smile, laugh, or chuckle?
      3. Punchiness - Is it short, sharp, and scroll-stopping?
      4. Virality Potential - Would people retweet or share it?
      5. Format - Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

    Auto-reject if:
      - It's written in question-answer format (e.g., "Why did ... " or "What happens when ... ")
      - It exceeds 280 characters
      - It reads like a traditional setup-punchline joke
      - Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., "Masterpieces of the auntie-uncle
      universe" or vague summaries)

    ### Respond ONLY in structured format:
      - evaluation: "approved" or "needs_improvement"
      - feedback: One paragraph explaining the strengths and weaknesses
    """)
  ]

  response = structuredEvaluationModel.invoke(messages)

  return {'evaluation':response.evaluation, 'feedback':response.feedback}


def optimizeTweet(state: tweetState):
  messages = [
    SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
    HumanMessage(content=f"""
    Improve the tweet based on this feedback:
    "{state['feedback']}"

    Topic: "{state['topic']}"
    Original Tweet:
    {state['tweet']}
    I
    Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
    """ )
  ]

  responce = optimizerModel.invoke(messages)

  iteration = state['iteration'] + 1

  return {'tweet': responce.content, 'iteration': iteration}


def routeEvaluation(state: tweetState):
  if state['evaluation'] == 'approved' or state['iteration'] >= state['maxIteration'] :
    return 'approved'
  else:
    return 'needsImprovement'

In [9]:
graph = StateGraph(tweetState)

graph.add_node('generate', generateTweet)
graph.add_node('evaluate', evaluateTweet)
graph.add_node('optimize', optimizeTweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')

graph.add_conditional_edges('evaluate', routeEvaluation, {'approved':END, 'needsImprovement':'optimize'})
graph.add_edge('optimize','evaluate')

workflow = graph.compile()


In [11]:
initialState = {
  'topic' : 'sfsjs',
  'iteration': 1,
  'maxIteration': 5
}

workflow.invoke(initialState)

{'topic': 'sfsjs',
 'tweet': "That moment when you realize your potato is having a mid-life crisis and suddenly, you’re the one in therapy. 🥔💔 Forget a PhD in procrastination—I'm majoring in Memeology! #MemeMasters #PrioritiesArePotatoes 🍟✨",
 'evaluation': 'needsImprovement',
 'feedback': "The tweet attempts to blend humor with originality by comparing a potato to a person experiencing a mid-life crisis and tying it into studying 'Memeology.' However, the humor doesn't quite hit the mark as it leans on a fairly common trope of personifying objects in quirky scenarios. While it's not in question-answer format and fits within the character limit, the joke lacks punchiness and specificity, leading to a diluted impact. It uses multiple hashtags, which can be distracting and doesn't contribute to its virality potential. A more focused take on the potato's 'crisis' could elevate the humor and make it more engaging.",
 'iteration': 5,
 'maxIteration': 5}